<img src="images/header.png" alt="Logo UCLM-ESII" align="right">

<br><br><br><br>
<h2><font color="#92002A" size=4>Trabajo Fin de Máster</font></h2>

<h1><font color="#6B001F" size=5>SERENDIPITY: Servicio web para la recomendacIón de playlists a partir de otra playlist</font></h1>
<h2><font color="#92002A" size=3>Parte 1 - Preparación de datos</font></h2>

<br>
<div style="text-align: right">
    <font color="#B20033" size=3><strong>Autor</strong>: <em>Miguel Ángel Cantero Víllora</em></font><br>
    <br>
    <font color="#B20033" size=3><strong>Directores</strong>: <em>José Antonio Gámez Martín</em></font><br>
    <font color="#B20033" size=3><em>Juan Ángel Aledo Sánchez</em></font><br>
    <br>
<font color="#B20033" size=3>Máster Universitario en Ingeniería Informática</font><br>
<font color="#B20033" size=2>Escuela Superior de Ingeniería Informática | Universidad de Castilla-La Mancha</font>

</div>

---

<br>


<a id="indice"></a>
<h2><font color="#92002A" size=5>Índice</font></h2>

<br>

* [1. Introducción](#section1)
* [2. Cambio de formato del dataset](#section2)
* [3. Modificaciones adicionales en el dataset](#section3)

<br>

---

In [1]:
import csv
import json
import os

from collections import defaultdict
from tqdm.notebook import tqdm as tqdm_nb
from zipfile import ZipFile

In [2]:
MPD_PATH = 'MPD'
MPD_TEST_PATH = 'MPD_TEST'
MPD_CSV_PATH = 'MPD_CSV'
MPD_SLICE_PREFIX = 'mpd.slice.'
PLSTRS_PREFIX = 'mpd.playlists-tracks.'
MPD_TEST_FILE = 'challenge_set.json'

ALBUMS_FILE = os.path.join(MPD_CSV_PATH,'mpd.albums.csv')
ARTISTS_FILE = os.path.join(MPD_CSV_PATH,'mpd.artists.csv')
TRACKS_FILE = os.path.join(MPD_CSV_PATH,'mpd.tracks.csv')
PLSTRS_FILE = os.path.join(MPD_CSV_PATH,'mpd.pls-tracks.csv')
PLSTARTS_FILE = os.path.join(MPD_CSV_PATH,'mpd.pls-artists.csv')
PLSALBUMS_FILE = os.path.join(MPD_CSV_PATH,'mpd.pls-albums.csv')
PLSINFO_FILE = os.path.join(MPD_CSV_PATH,'mpd.playlists-info.csv')
PLSTESTINFO_FILE = os.path.join(MPD_CSV_PATH,'mpd.playlists-info-test.csv')

---

<br>


<a id="section1"></a>
## <font color="#92002A">1 - Introducción</font>
<br>

En esta libreta, vamos a realizar una serie de cambios en el conjunto de playlists que obtuvimos en el proyecto _[Generación automática de playlist de canciones mediante técnicas de minería de datos](https://github.com/miguelangelcv/TFG-GeneracionPlaylists)_. Lo primero que haremos será convertirlos de su formato original, *json*, a formato *csv*. Este cambio lo realizamos para reducir su tamaño y poder almacenarlo en memoria y que también sea compatible con librerías como _[numpy](https://numpy.org/)_ o _[pandas](https://pandas.pydata.org/)_. También realizaremos una serie de modificaciones adicionales para eliminar información innecesaria.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<br>


<a id="section2"></a>
## <font color="#92002A">2 - Cambio de formato del dataset</font>
<br>


Este proceso generará varios ficheros *csv* que contendrán distinta información, ya sea sobre las playlists, artistas, pistas, etc. Lo hacemos de dicha forma para que la información este mejor clasificada y sea más fácil trabajar con ella.

Para realizar el cambio de formato del dataset, hemos creado una serie de funciones que nos ayudaran a leer y transformar los datos:

* `json_file_reader`: Esta función permite leer los ficheros que componen el conjunto de datos, ya sea un archivo *json* o un archivo *zip* que lo contenga.
* `jsonds_to_csvds`: Se encarga procesar todos los ficheros correspondientes al conjunto de datos que contiene las playlists y transformarlos a formato *csv* y repartir la información en varios ficheros. 
* `jsonds_to_csvds_test`: Realiza el mismo proceso que la función anterior, pero con el conjunto de datos que emplearemos como conjunto de prueba (o test).

<br>

In [3]:
# Función que nos permite leer un archivo .json comprimido o sin comprimir
# y devuelve un diccionario con su contenido
def json_file_reader(file_path):
    """
    :param file_path: Ruta del fichero a leer.
    :return results: Diccionario con los datos leidos del fichero JSON.
    """
    _ , file_extension = os.path.splitext(file_path)

    # Fichero comprimido
    if file_extension == '.zip':
        with ZipFile(file_path,'r') as zip_file:
            with zip_file.open(zip_file.namelist()[0]) as json_file:
                json_data = json.load(json_file)
    # Fichero sin comprimir
    elif file_extension == '.json':
        with open(file_path, "r") as json_file:
            json_data = json.load(json_file)            
    # En caso de que sea otra extensión, devolvemos un diccionario vacío
    else:
        json_data = {}            
    
    return json_data

In [4]:
# Función que se encarga de convertir el dataset que contiene 
# 1 millón de playlists a formato CSV
def jsonds_to_csvds(json_ds_path,csv_ds_path):
    """
    :param json_ds_path: Ruta donde se encuentra el conjunto de datos en formato JSON.
    :param csv_ds_path: Ruta donde almacenaremos el nuevo conjunto de datos en formato CSV.
    """
    if not os.path.isdir(csv_ds_path):
        os.mkdir(csv_ds_path)    
    
    files = []
    tracks_dict = defaultdict(dict)

    for file in os.listdir(json_ds_path):
        if file.startswith(MPD_SLICE_PREFIX):
            files.append(os.path.join(json_ds_path,file))

    plstrs_fieldnames = ['pid','pos','track_uri']
    tracks_fieldnames = ['track_name', 'track_uri', 'duration_ms', 'artist_name', 
                         'artist_uri', 'album_name', 'album_uri']
    plsinfo_fieldnames = ['pid','name','collaborative','modified_at',
                          'num_albums','num_tracks', 'num_followers',
                          'num_edits','duration_ms','num_artists','description']

    with open(PLSINFO_FILE,'w',newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=plsinfo_fieldnames,delimiter=',')
        writer.writeheader()

    print("[1/2] Lectura de datos en JSON:")
    for file in tqdm_nb(files):
        file_name , _ = os.path.splitext(file)
        portion = file_name.split('.')[-1]
        csv_pltrs_file = os.path.join(csv_ds_path, f"{PLSTRS_PREFIX}{portion}.csv")
        row_list = []

        with open(PLSINFO_FILE,'a',encoding='utf8',newline='') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=plsinfo_fieldnames,
                                    delimiter=',',quoting=csv.QUOTE_MINIMAL)
            for pl in json_file_reader(file)['playlists']:
                tracks_list = pl.pop('tracks')
                writer.writerow(pl)
                for track in tracks_list:
                    pos = track.pop('pos')
                    row = {'pid': pl['pid'], 'pos': pos,
                           'track_uri' : track['track_uri']}
                    row_list.append(row)
                    tracks_dict[track['track_uri']] = track

        with open(csv_pltrs_file,'w',newline='') as csv_tracks_file:
            writer_tracks = csv.DictWriter(csv_tracks_file, fieldnames=plstrs_fieldnames)
            writer_tracks.writeheader()
            for row in row_list:
                writer_tracks.writerow(row)
    
    print("[2/2] Almacenamiento de datos en CSV:")
    with open(TRACKS_FILE,'w',newline='', encoding='utf8') as csv_tracks_file:
        writer_tracks = csv.DictWriter(csv_tracks_file, fieldnames=tracks_fieldnames)
        writer_tracks.writeheader()
        pbar = tqdm_nb(total=len(tracks_dict))
        for track in tracks_dict.values():
            writer_tracks.writerow(track)
            pbar.update(1)

In [5]:
# Función que se encarga de convertir el conjunto de datos (test)
# a formato CSV
def jsonds_to_csvds_test(json_ds_path,csv_ds_path):
    """
    :param json_ds_path: Ruta donde se encuentra el conjunto de datos en formato JSON.
    :param csv_ds_path: Ruta donde almacenaremos el nuevo conjunto de datos en formato CSV.
    """
    if not os.path.isdir(csv_ds_path):
        os.mkdir(csv_ds_path)

    if MPD_TEST_FILE in os.listdir(json_ds_path):
        plstrs_fieldnames = ['pid','pos','track_uri']
        plsinfo_fieldnames = ['pid','name','num_holdouts','num_samples',
                              'num_tracks', 'description']
        row_list = []
        
        with open(PLSTESTINFO_FILE, 'w',newline='',encoding='utf8') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=plsinfo_fieldnames,
                                    delimiter=',',quoting=csv.QUOTE_MINIMAL)
            writer.writeheader()
            test_playlists = json_file_reader(os.path.join(json_ds_path,MPD_TEST_FILE))['playlists']
            for pl in test_playlists:
                tracks_list = pl.pop('tracks')
                writer.writerow(pl)
                
                for track in tracks_list:
                    pos = track.pop('pos')
                    row = {'pid': pl['pid'], 'pos': pos,
                           'track_uri' : track['track_uri']}
                    row_list.append(row)
        
        plstrs_test_path = os.path.join(MPD_CSV_PATH,"{}test.csv".format(PLSTRS_PREFIX))
        with open(plstrs_test_path, 'w', newline='', encoding='utf8') as csv_tracks_file:
            writer_tracks = csv.DictWriter(csv_tracks_file, fieldnames=plstrs_fieldnames)
            writer_tracks.writeheader()
            for row in row_list:
                writer_tracks.writerow(row)    
    else:
        print("ERROR: Fichero del conjunto de prueba no encontrado")

<br>

Una vez definidas las funciones, realizamos el proceso de conversión.

<br>

<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
__Importante__: Este proceso puede durar entre 15 y 20.
</div>

<br>

In [6]:
jsonds_to_csvds(MPD_PATH,MPD_CSV_PATH)

In [7]:
jsonds_to_csvds_test(MPD_TEST_PATH,MPD_CSV_PATH)

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<br>


<a id="section3"></a>
## <font color="#92002A">3 - Modificaciones adicionales en el dataset</font>
<br>

Con el conjunto de datos ya convertido en formato *csv*, vamos a realizar una serie de cambios con los que crearemos un índice propio, distinto al de *Spotify*, para identificar los distintos elementos que conforman nuestro dataset y eliminar información innecesaria o repetida, como los prefijos de los identificadores de pistas, artistas o álbumes. También extraeremos información a otros ficheros *csv* para tener la información mejor ordenada.

<br>

<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
__Importante__: Este proceso puede tardar hasta 10 minutos en completarse.
</div>

<br>

In [8]:
import pandas as pd

df_tracks = pd.read_csv(TRACKS_FILE)

<br>

<u>**Paso 1**</u><br> Creamos un conjunto de datos que contendrá la información relativa a los artistas (identificador, nombre e identificador de *Spotify* sin prefijo):

In [9]:
df_artists = df_tracks[['artist_name', 'artist_uri']].copy()
df_artists.drop_duplicates(subset=['artist_uri'],inplace=True)
df_artists.reset_index(drop=True, inplace=True)
df_artists.index.name = 'artist_pid'
df_artists['artist_id'] = df_artists['artist_uri'].apply(lambda x: x.split(':')[-1])

<br>

<u><strong>Paso 2</strong></u><br> Creamos un conjunto de datos que contendrá la información relativa a los álbumes (identificador, nombre, artista al que pertenece e identificador de *Spotify* sin prefijo):

In [10]:
df_albums = df_tracks[['album_name', 'album_uri', 'artist_uri']].copy()
df_albums.drop_duplicates(subset=['album_uri'],inplace=True)
df_albums.reset_index(drop=True, inplace=True)
df_albums.index.name = 'album_pid'

<br>

<u><strong>Paso 3</strong></u><br> Eliminamos del conjunto de pistas el nombre del artista y del álbum:

In [11]:
df_tracks.drop(columns=['artist_name','album_name'],inplace=True)
df_tracks.drop_duplicates(subset='track_uri',inplace=True)
df_tracks.reset_index(drop=True, inplace=True)

<br>

<u><strong>Paso 4</strong></u><br> 
Sustituimos del conjunto de pistas los identificadores de *Spotify* por nuestros propios identificadores:

In [12]:
df_tracks = pd.merge(df_tracks, df_artists.reset_index()[['artist_pid','artist_uri']], on=['artist_uri']) \
                .drop(columns=['artist_uri'])
df_tracks = pd.merge(df_tracks, df_albums.reset_index()[['album_pid','album_uri']], on=['album_uri']) \
                .drop(columns=['album_uri'])

df_tracks.index.name = 'track_pid'

df_tracks['track_id'] = df_tracks['track_uri'].apply(lambda x: x.split(':')[-1])

<br>

<u><strong>Paso 5</strong></u><br> 
Sustituimos del conjunto de álbumes los identificadores de *Spotify* por nuestros propios identificadores:

In [13]:
df_albums = pd.merge(df_albums, df_artists.reset_index()[['artist_pid','artist_uri']], on=['artist_uri']) \
                .drop(columns=['artist_uri'])
df_albums.index.name = 'album_pid'

df_albums['album_id'] = df_albums['album_uri'].apply(lambda x: x.split(':')[-1])
df_albums.drop(columns=['album_uri'], inplace = True)
df_artists.drop(columns=['artist_uri'], inplace = True)

<br>

<u><strong>Paso 6</strong></u><br> 
En el conjunto que contiene qué pistas contiene cada playlists, sustituimos los identificadores de *Spotify* para las playlists y pistas por nuestros propios identificadores y los unimos en un único fichero:

In [14]:
# Función empleada para leer un dataframe que ha sido almacenado
# en varios ficheros
def read_dataset_multifile(ds_prefix, folder=os.curdir):
    """
    :param ds_prefix: Prefijo de los ficheros a leer.
    :param folder: Directorio donde se encuentran los ficheros.
    :return: Dataframe resultante de leer los ficheros.
    """
    list_df = []
    
    for file_name in os.listdir(folder):
        if file_name.startswith(ds_prefix):
            file_path = os.path.join(folder, file_name)
            df_temp = pd.read_csv(file_path)
            list_df.append(df_temp)
            
    return pd.concat(list_df, axis=0, ignore_index=True)

In [15]:
df_plstrs = read_dataset_multifile(PLSTRS_PREFIX,MPD_CSV_PATH)

trackid_map_dict = df_tracks[['track_uri']].to_dict()['track_uri']
trackid_map_dict = {v: k for k, v in trackid_map_dict.items()}

df_plstrs["track_pid"] = df_plstrs["track_uri"].map(trackid_map_dict)
df_plstrs.drop(columns=['track_uri'], inplace=True)
df_tracks.drop(columns=['track_uri'], inplace=True)
df_plstrs.sort_values(['pid', 'pos'],inplace=True)
df_plstrs.set_index('pid',inplace=True)
df_plstrs.index.name = 'pl_pid'

In [16]:
for file_name in os.listdir(MPD_CSV_PATH):
    if file_name.startswith(PLSTRS_PREFIX):
        os.remove(os.path.join(MPD_CSV_PATH,file_name))

<br>

<u><strong>Paso 7</strong></u><br> 
Adicionalmente creamos dos nuevos conjuntos, uno que contiene que artistas aparecen en cada playlist y otro que contiene que álbumes contiene cada playlist, cada uno de ellos junto al número de apariciones de dichos elementos:

In [17]:
# Apariciones de artistas en las playlits
df_plsarts = pd.merge(df_plstrs.reset_index(), 
                      df_tracks.reset_index()[['track_pid', 'artist_pid']], on=['track_pid'])
df_plsarts.drop(columns=['track_pid','pos'], inplace=True)
df_plsarts.sort_values(by=['pl_pid','artist_pid'], inplace=True)
df_plsarts = df_plsarts.groupby(['pl_pid','artist_pid']).size().to_frame(name = 'artist_count').reset_index()
df_plsarts.set_index('pl_pid',inplace=True)

In [18]:
# Apariciones de álbumes en las playlits
df_plsalbums = pd.merge(df_plstrs.reset_index(), 
                        df_tracks.reset_index()[['track_pid','album_pid']], on=['track_pid'])
df_plsalbums.drop(columns=['track_pid','pos'], inplace=True)
df_plsalbums.sort_values(by=['pl_pid','album_pid'], inplace=True)
df_plsalbums = df_plsalbums.groupby(['pl_pid','album_pid']).size().to_frame(name = 'album_count').reset_index()
df_plsalbums.set_index('pl_pid',inplace=True)

<br>

<u><strong>Paso 8</strong></u><br> 

Modificamos el nombre de la columna relativa a los identificadores por el de _pl_pid_ y establecemos el tipo de datos `int` de cada columna de tipo numérico en los dataframes que contienen la información relativas a las playlists:

In [19]:
plinfo_dtypes = {'modified_at' : int, 'num_albums': int, 'num_tracks': int, 'num_followers' : int,
                 'num_edits': int, 'duration_ms' : int, 'num_artists': int}

df_plsinfo = pd.read_csv(PLSINFO_FILE, dtype=plinfo_dtypes,index_col=0)
df_plsinfo.index.name = 'pl_pid'

In [20]:
plinfotest_dtypes = {'num_holdouts' : int, 'num_samples' : int, 'num_tracks' : int}

df_plsinfotest = pd.read_csv(PLSINFO_FILE, dtype=plinfotest_dtypes,index_col=0)
df_plsinfotest.index.name = 'pl_pid'

<br><br>

Realizadas las correspondientes modificaciones, procedemos a almacenar los archivos que conforman el conjunto de datos de playlists:

In [21]:
df_plsarts.to_csv(PLSTARTS_FILE)
df_artists.to_csv(ARTISTS_FILE)
df_albums.to_csv(ALBUMS_FILE)
df_tracks.to_csv(TRACKS_FILE)
df_plsinfo.to_csv(PLSINFO_FILE)
df_plsinfotest.to_csv(PLSTESTINFO_FILE)
df_plstrs.to_csv(PLSTRS_FILE)
df_plsalbums.to_csv(PLSALBUMS_FILE)

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-graduation-cap" aria-hidden="true" style="color:#92002A"></i> </font></div>